In [ ]:
# !pip install --upgrade transformers

In [ ]:
# !pip install auto-round 

In [ ]:
# !MAX_JOBS=8 pip install flash-attn --no-build-isolation

In [1]:
import os
import torch
from auto_round import AutoRound
from huggingface_hub import HfApi, create_repo, notebook_login, get_token
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


PyTorch Version: 2.8.0+cu126
CUDA Available: True
CUDA Version: 12.6
GPU Name: NVIDIA RTX A6000
VRAM: 47.5 GB


In [4]:
MODEL_ID = "Qwen/Qwen3-VL-2B-Instruct"
HF_USER = "Vishva007"
OUTPUT_BASE_DIR = "./AutoRound"

In [5]:
notebook_login()

In [6]:
# "AWQ",            # Best for Nvidia GPUs (vLLM, TGI)
# "GPTQ",           # Good compatibility
# "AutoRound",      # Intel default format (Requires auto-round lib to run)
# "GGUF",           # For llama.cpp / Olama

TARGET_FORMATS = "AWQ"

use_fp16 = TARGET_FORMATS in ["AWQ", "GPTQ"]


# High-End GPU Tuning Parameters (A40/A6000/L40)
TUNING_CONFIG = {
    "group_size": 128,
    "sym": True,
    "iters": 50,           # High accuracy (Production grade)
    "nsamples": 128,         # More calibration data
    "batch_size": 2,        # Faster on 48GB VRAM
    "seqlen": 2048,
    "low_gpu_mem_usage": False,   # Keep on GPU for speed
    "enable_torch_compile": True, # JIT acceleration
    "quant_nontext_module": False, # Keep Vision Tower in BF16 (Crucial for VLM accuracy)
}



In [7]:
def push_to_hub(local_dir, repo_name, token):
    """Creates repo and uploads folder to Hugging Face."""
    full_repo_id = f"{HF_USER}/{repo_name}"
    print(f"\n[Hub] Pushing {local_dir} to {full_repo_id}...")
    
    try:
        api = HfApi()
        create_repo(full_repo_id, repo_type="model", exist_ok=True, private=False, token=token)
        
        api.upload_folder(
            folder_path=local_dir,
            repo_id=full_repo_id,
            repo_type="model",
            token=token
        )
        print(f"[Hub] ✅ Successfully uploaded: https://huggingface.co/{full_repo_id}")
    except Exception as e:
        print(f"[Hub] ❌ Error uploading: {e}")

In [8]:
import tokenize


model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_ID, 
    dtype=torch.float16 if use_fp16 else "auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

tokenizer = processor.tokenizer


In [9]:

ar = AutoRound(
        model=model,
        tokenizer=tokenizer,
        processor=processor,
        scheme="W4A16",
        model_dtype="fp16" if use_fp16 else "bf16",
        **TUNING_CONFIG
    )

2026-02-02 02:48:40 INFO autoround.py L158: using MLLM mode for multimodal model.
2026-02-02 02:48:40 INFO base.py L391: using torch.float16 for quantization tuning


In [10]:
base_name = MODEL_ID.split("/")[-1]

if TARGET_FORMATS == "AWQ":
    dir_suffix = "W4A16-AWQ"
    format_arg = "auto_awq"
elif TARGET_FORMATS == "GPTQ":
    dir_suffix = "W4A16-GPTQ"
    format_arg = "auto_gptq"
else: # AutoRound
    dir_suffix = "W4A16-AutoRound"
    format_arg = "auto_round"

save_dir = os.path.join(OUTPUT_BASE_DIR, dir_suffix)


ar.quantize_and_save(
    format=format_arg, 
    output_dir=save_dir, 
    inplace=False
)

2026-02-02 02:48:52 INFO base.py L1729: start to cache block inputs
cache block inputs: 100%|██████████| 128/128 [00:01<00:00, 93.63it/s] 
2026-02-02 02:48:58 INFO base.py L1744: caching done
Quantizing model.language_model.layers.0:   0%|          | 0/28 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:829: UserWarning: Flash Attention defaults to a non-deterministic algorithm. To explicitly enable determinism call torch.use_deterministic_algorithms(True, warn_only=False). (Triggered internally at /pytorch/aten/src/ATen/native/transformers/cuda/attention_backward.cu:110.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
quantized 7/7 layers in the block, loss iter 0: 0.001456 -> iter 27: 0.000716,'peak_ram': 6.47GB, 'peak_vram': 4.07GB
Quantizing done: 100%|██████████| 28/28 [01:15<00:00,  2.69s/it]                          
2026-02-02 02:50:13 INFO device.py L1430: 'peak_ram': 7.98GB, 'peak_vram': 4

(Qwen3VLForConditionalGeneration(
   (model): Qwen3VLModel(
     (visual): Qwen3VLVisionModel(
       (patch_embed): Qwen3VLVisionPatchEmbed(
         (proj): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
       )
       (pos_embed): Embedding(2304, 1024)
       (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
       (blocks): ModuleList(
         (0-23): 24 x Qwen3VLVisionBlock(
           (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
           (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
           (attn): Qwen3VLVisionAttention(
             (qkv): Linear(in_features=1024, out_features=3072, bias=True)
             (proj): Linear(in_features=1024, out_features=1024, bias=True)
           )
           (mlp): Qwen3VLVisionMLP(
             (linear_fc1): Linear(in_features=1024, out_features=4096, bias=True)
             (linear_fc2): Linear(in_features=4096, out_features=1024, bias=True)
             (act_fn): GELUTanh()
           

In [11]:
hf_token = get_token()

if hf_token:
    push_to_hub(save_dir, f"{base_name}-W4A16-AutoRound-{TARGET_FORMATS}", hf_token)
else:
    print("❌ No HF token found. Skipping upload.")



[Hub] Pushing ./AutoRound/W4A16-AWQ to Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound-AWQ...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[Hub] ✅ Successfully uploaded: https://huggingface.co/Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound-AWQ
